In [ ]:
import numpy as np
import pandas as pd

TOP_N = 100

# Convert .csv dataset to DataFrame
data = pd.read_csv("Twitch_game_data.csv", sep=',')

# Clean data
    # Remove " hours" from Hours_Streamed column
    # Convert Hours_Streamed column to int
data["Hours_Streamed"] = data["Hours_Streamed"].map(lambda x : x.rstrip(" hours"))
data["Hours_Streamed"] = pd.to_numeric(data["Hours_Streamed"])

    # Remove ranks past TOP_N
data = data.loc[data['Rank'] <= TOP_N]

print("Number of Unique Games: ",data["Game"].unique().size)
data.head(10)